#### Import the relevant librries

In [174]:
import pandas as pd
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.optimizers import Adam
import pickle
import time, datetime, os
#%reload_ext tensorboard

####  Set tensorboard and import final_dataset as stats.


In [175]:
NAME = "264x32x16x1_MSE_sv{}".format(int(time.time()))
#log_dir = "logs/fit/" + NAME
#tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
stats = pd.read_csv(r'Data\final_dataset.csv')
stats['Results'] = stats['Results'].astype('float32')
stats

,GAME_DATE,HOME,AWAY,AST _ewm10,BLK _ewm10,DREB _ewm10,FG_PCT _ewm10,FG3_PCT _ewm10,FG3A _ewm10,FG3M _ewm10,...,OPP_PTS_OFF_TOV_ewm5ha,OPP_PTS_PAINT_ewm5ha,PTS_2ND_CHANCE_ewm5ha,PTS_FB_ewm5ha,PTS_OFF_TOV_ewm5ha,PTS_PAINT_ewm5ha,Score,Odds1,Odds2,Results
0,28/10/2008,BOS,CLE,-0.053016,0.008363,0.014856,-0.000706,0.000401,0.096037,0.040449,...,0.433684,-0.361584,0.104689,-0.785678,-0.016826,-0.015002,90:85,1.36,3.15,1.0
1,28/10/2008,CHI,MIL,-0.001185,-0.004342,-0.058237,-0.000422,-0.001997,-0.150581,-0.082637,...,0.120543,-0.110391,0.363791,0.588124,-0.552351,1.246822,108:95,1.36,3.20,1.0
2,28/10/2008,LAL,POR,-0.081911,0.001941,0.027199,-0.001031,-0.001619,0.030843,-0.018690,...,-0.056882,0.159378,-0.498689,-3.364871,-0.313756,-0.855125,96:76,1.27,3.85,1.0
3,29/10/2008,DET,IND,0.052314,0.020826,-0.020202,0.000044,0.000531,0.045038,0.029248,...,-0.008929,1.189835,0.620750,2.002133,1.740860,0.471794,100:94,1.13,5.75,1.0
4,29/10/2008,GSW,NOP,-0.026685,0.038596,0.004095,0.000155,-0.000702,-0.142057,-0.061073,...,-0.777967,0.603658,-0.257766,0.099136,-0.461950,-0.713632,103:108,3.35,1.32,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14295,10/04/2022,NYK,TOR,0.035821,0.015852,0.037074,-0.000346,0.000882,0.100520,0.063328,...,-0.236713,-0.154263,-0.810998,-1.268378,-0.856542,-0.941926,105:94,1.96,1.89,1.0
14296,10/04/2022,ORL,MIA,-0.004794,0.036993,0.051106,-0.001918,-0.002143,0.085899,-0.050124,...,-0.372876,0.077874,0.597325,-0.290637,-0.696616,0.265701,125:111,3.41,1.34,1.0
14297,10/04/2022,PHI,DET,0.045087,-0.011308,0.016990,0.001253,0.001721,-0.060110,0.050316,...,0.554088,-0.275713,0.322300,-0.649111,-0.162648,1.015744,118:106,1.37,3.24,1.0
14298,10/04/2022,PHX,SAC,0.080521,-0.009226,0.047192,0.000443,0.000586,0.027357,0.024687,...,0.285610,-0.141892,0.227124,1.075718,0.597625,0.317031,109:116,1.19,5.16,0.0


#### Slice stats into X and y. Split the data into training and test set (80/20) and normalize the data

In [176]:
X = stats.iloc[:,3:-4]
y = stats.iloc[:,-1:]
random_state = 12
#split in order, we don't want our model to predict outcomes with data that have not happened yet.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle = False, random_state=random_state)
X_train = tf.keras.utils.normalize(X_train, axis=1)
X_test = tf.keras.utils.normalize(X_test, axis=1)
stats['Results'] = stats['Results'].astype('float32')

### Define the custom loss functions

##### BCE: Binary-cross entropy with added parameters to optimize our return on investment.

$L(y\hat, y) = -\sum\limits_{i=1}^{m} (Odds_{1i}*y_{i}-1) * log(Odds_{1i}*\hat y_{i}-1) + (Odds_{2i}(1 - y_{i})-1) * log(Odds_{2i}*(1- \hat y_{i}) -1)$

Source: https://www.vantage-ai.com/en/blog/beating-the-bookies-with-machine-learning

We will try to fit these loss function in a near future.

##### MSE: Mean Squared Error with odds decorrelation

$L(p\hat, y)= \frac{1}{N} \sum\limits_{i=1}^{N}(\hat p_{i} - y_{i})^{2} - C * (\hat p_{i} - \frac {1}{Odds_{i}})^{2}$

The first part $(\hat p_{i} - y_{i})^{2}$ is the Mean Squared Error, the difference between the predicted outcome versus the actual aoutcome. For the second part, C is a constant that determines the significance of the decorrelation effect. $(\hat p_{i} - \frac {1}{Odds_{i}})^{2}$, $\hat p_{i}$ is the predicted probability for the home team and $\frac {1}{Odds_{i}}$ is the probability that the bookmakers gives to the home team victory.

Source: Hubáček, Ondřej, Gustav Šourek, and Filip Železný. "Exploiting sports-betting market using machine learning." International Journal of Forecasting 35.2 (2019): 783-796.

Keras only allow the parameters y_pred and y_true in the loss functions, to make our custom loss function we pass a wrapper function with our extra parameter, the odds.


In [177]:
odds1 = stats.iloc[:, -3:-2].values
odds1_prob = 1/odds1

def coef_bce(y_pred,y_true, odds1, odds2):
    return -1 * K.mean((odds1 * y_true -1) * K.log(odds1 * y_pred -1) + (odds2 * (1- y_true) -1) * K.log(odds2 * (1-y_pred)-1))

def bce_custom(odds1, odds2):
    def bce(y_pred, y_true):
        return coef_bce(y_pred, y_true, odds1, odds2)
    return bce

#loss_bce = bce_custom(odds1, odds2)


def coef_mse(y_pred, y_true, odds_game ):
    #mse = K.square(y_pred - y_true)  #squared difference
    #odds = -0.10 * K.square(y_pred - odds_game)   
    #loss = K.mean(mse - odds , axis=-1) #mean over last dimension
    #return loss
    return K.mean(K.square(y_pred - y_true) +0.2 * K.square(y_pred- odds_game), axis=-1)

def mse_custom(odds_game):
    def mse( y_pred, y_true):
        return coef_mse(y_pred, y_true, odds_game)
    return mse

loss_mse = mse_custom(odds1_prob)


def mse_simple(y_true, y_pred):
    mse = math_ops.squared_difference(y_pred, y_true) 
    loss = K.mean(mse, axis=-1)
    return loss

### Model Architecture:
After configuring GridSearch, the best parameters where:
- layers: 64x32 // Input is 264(n. of features) and output is 1. So (264 x 64 x 32 x 1)
- epochs: 5
- batch size: 3
- regularisation: l2(0.01)
- activation function: ReLU except output layer (sigmoid)
- optimizer: Adam

In [178]:
#opt = tf.keras.optimizers.Adam(clipnorm=0.3)
#bias_regularizer=l2(0.01)

model = Sequential()
#model.add(tf.keras.layers.Flatten())
model.add(Dense(264, input_dim=264, kernel_initializer='normal', activation='relu',bias_regularizer=l2(0.01)))
model.add(Dense(64, kernel_initializer='normal', activation='relu',bias_regularizer=l2(0.01)))
model.add(Dense(32, kernel_initializer='normal', activation='relu',bias_regularizer=l2(0.01)))
model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
model.compile(optimizer='Adam', loss='MeanSquaredError', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size= 1, epochs=5, validation_data=(X_test, y_test))

Epoch 1/5
11440/11440 [==============================] - 20s 2ms/step - loss: 0.2193 - accuracy: 0.6490 - val_loss: 0.2301 - val_accuracy: 0.6259
Epoch 2/5
11440/11440 [==============================] - 19s 2ms/step - loss: 0.2145 - accuracy: 0.6645 - val_loss: 0.2298 - val_accuracy: 0.6318
Epoch 3/5
11440/11440 [==============================] - 20s 2ms/step - loss: 0.2125 - accuracy: 0.6713 - val_loss: 0.2284 - val_accuracy: 0.6346
Epoch 4/5
11440/11440 [==============================] - 19s 2ms/step - loss: 0.2105 - accuracy: 0.6723 - val_loss: 0.2319 - val_accuracy: 0.6241
Epoch 5/5
11440/11440 [==============================] - 21s 2ms/step - loss: 0.2089 - accuracy: 0.6765 - val_loss: 0.2261 - val_accuracy: 0.6339


In [179]:
#val_loss, val_acc = model.evaluate(X_test, y_test)


### Retrieve the information attached to the predictions and  store the predictions

We attach the prediction with the outcome. We also retrieve the infomration regarding the game such as the data, the teams, the score and the odds so we can later perform an Error analysis. We finally store the dataframe in a csv file.

In [180]:
prediction = model.predict(X_test[:])
label = y_test[:]
prediction_l = prediction.tolist()
pred =pd.DataFrame(prediction_l, columns=['prediction'])

In [181]:
stats.reset_index
index = stats[['GAME_DATE', 'HOME', 'AWAY','Odds1', 'Odds2', 'Score']]
data = label.join(index)
data.reset_index(inplace=True)
pred_data = data.join(pred)
pred_data.to_csv('predicted_data.csv')
pred_data

,index,Results,GAME_DATE,HOME,AWAY,Odds1,Odds2,Score,prediction
0,11440,1.0,23/10/2019,ORL,CLE,1.20,4.92,94:85,0.664552
1,11441,1.0,23/10/2019,PHI,BOS,1.42,3.01,107:93,0.390108
2,11442,1.0,23/10/2019,PHX,SAC,1.89,1.96,124:95,0.546155
3,11443,0.0,23/10/2019,POR,DEN,2.02,1.84,100:108,0.688613
4,11444,1.0,23/10/2019,SAS,NYK,1.16,5.56,120:111,0.725813
...,...,...,...,...,...,...,...,...,...
2855,14295,1.0,10/04/2022,NYK,TOR,1.96,1.89,105:94,0.460038
2856,14296,1.0,10/04/2022,ORL,MIA,3.41,1.34,125:111,0.256844
2857,14297,1.0,10/04/2022,PHI,DET,1.37,3.24,118:106,0.783105
2858,14298,0.0,10/04/2022,PHX,SAC,1.19,5.16,109:116,0.738342


### GridSearchCV

We use the GridSearchCV from sklearn to find the parameters that best optimize our model.

In [10]:
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
def model_nn(units='8', units2='8', drop='0,1', bias = 0.01, bias2 = 0.01, bias3 = 0.01):
    model = Sequential()
    #model.add(tf.keras.layers.Flatten())
    model.add(Dense(264, input_dim=264, kernel_initializer='normal', activation='relu',bias_regularizer=l2(bias)))
    model.add(Dense(units=units, kernel_initializer='normal', activation='relu',bias_regularizer=l2(bias2)))
    model.add(Dense(units=units2, kernel_initializer='normal', activation='relu',bias_regularizer=l2(bias3)))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))

    model.compile(optimizer='adam', loss='MeanSquaredError', metrics=['accuracy'])
    #model.fit(X_train, y_train, batch_size= 5, epochs=10, validation_data=(X_test, y_test), callbacks=[tensorboard_callback])
    return model

model = KerasClassifier(build_fn=model_nn)
params={'bias':[0.01, 0.02, 0.03, 0.04],
        'bias2':[0.01, 0.02, 0.03, 0.04],
        'bias3':[0.01, 0.02, 0.03, 0.04],
        'units':[64],
        'units2':[32],
        'batch_size':[3], 
        'nb_epoch':[5]

        }
gs=GridSearchCV(estimator=model, param_grid=params, cv=10)
# now fit the dataset to the GridSearchCV object. 
gs = gs.fit(X_train, y_train)

Using TensorFlow backend.


382/382 [==============================] - 0s 389us/step - loss: 0.2156 - accuracy: 0.6486


382/382 [==============================] - 0s 390us/step - loss: 0.2205 - accuracy: 0.6547


3432/3432 [==============================] - 2s 656us/step - loss: 0.2223 - accuracy: 0.6421


3432/3432 [==============================] - 2s 712us/step - loss: 0.2230 - accuracy: 0.6459


3432/3432 [==============================] - 3s 735us/step - loss: 0.2225 - accuracy: 0.6445


382/382 [==============================] - 0s 437us/step - loss: 0.2259 - accuracy: 0.6486


3432/3432 [==============================] - 3s 873us/step - loss: 0.2230 - accuracy: 0.6392


3814/3814 [==============================] - 3s 866us/step - loss: 0.2222 - accuracy: 0.6467


In [92]:
best_params=gs.best_params_
accuracy=gs.best_score_
accuracy
best_params

{'batch_size': 3, 'bias': 0.01, 'nb_epoch': 5, 'units': 64, 'units2': 32}

In [43]:
#%tensorboard --logdir logs/fit